### Desafio Data Science - oitavo módulo &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; by Flávio

## Descubra as melhores notas de matemática do ENEM 2016

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from pandas_profiling import ProfileReport
from tpot import TPOTRegressor

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

**Recomendo uso deste Profiling, neste [artigo](https://towardsdatascience.com/exploratory-data-analysis-with-pandas-profiling-de3aae2ddff3) você pode ver outras possibilidades**

In [3]:
ProfileReport(df_test) 

C:\ProgramData\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,47
Number of observations,4576
Total Missing (%),7.6%
Total size in memory,1.6 MiB
Average record size in memory,376.0 B
Numeric,17
Categorical,14
Boolean,7
Date,0
Text (Unique),1
Rejected,8


In [4]:
# drop colunas com mais de 50% de dados faltantes
# e correlatas
df_test.drop(['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'IN_CEGUEIRA', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_IDOSO', 
              'TP_PRESENCA_CH','TP_PRESENCA_LC','Q027'], axis=1, inplace=True)

In [5]:
features = df_test.columns.to_list()
features.append('NU_NOTA_MT')
features

['NU_INSCRICAO',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'IN_TREINEIRO',
 'IN_BAIXA_VISAO',
 'IN_SURDEZ',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'TP_PRESENCA_CN',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 'Q047',
 'NU_NOTA_MT']

In [6]:
# ajuste no DF treino
df_train = df_train[features]

In [7]:
# vamos fazer o profiling destes dados e ver as correlações com NU_NOTA_MT
ProfileReport(df_train) 

C:\ProgramData\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,39
Number of observations,13730
Total Missing (%),6.6%
Total size in memory,4.1 MiB
Average record size in memory,312.0 B
Numeric,18
Categorical,13
Boolean,6
Date,0
Text (Unique),1
Rejected,1


**Limpando mais um pouco os dados de treino**

In [8]:
more_to_drop = ['TP_PRESENCA_CN', 'IN_GESTANTE', 'IN_SABATISTA', 'IN_SURDEZ', 'IN_BAIXA_VISAO', 'IN_TREINEIRO', 'TP_ESCOLA',
              'NU_NOTA_REDACAO', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'TP_NACIONALIDADE',
              'CO_PROVA_LC', 'CO_PROVA_MT']

In [9]:
df_train.drop(columns=more_to_drop, inplace=True)

In [10]:
df_train.isna().mean()

NU_INSCRICAO         0.000000
NU_IDADE             0.000000
TP_SEXO              0.000000
TP_COR_RACA          0.000000
TP_ST_CONCLUSAO      0.000000
TP_ANO_CONCLUIU      0.000000
NU_NOTA_CN           0.246832
NU_NOTA_CH           0.246832
NU_NOTA_LC           0.261981
TP_LINGUA            0.000000
TP_STATUS_REDACAO    0.261981
NU_NOTA_COMP1        0.261981
NU_NOTA_COMP2        0.261981
NU_NOTA_COMP3        0.261981
NU_NOTA_COMP4        0.261981
NU_NOTA_COMP5        0.261981
Q001                 0.000000
Q002                 0.000000
Q006                 0.000000
Q024                 0.000000
Q025                 0.000000
Q026                 0.000000
Q047                 0.000000
NU_NOTA_MT           0.261981
dtype: float64

In [11]:
# drop linhas que contem nulos
df_train.dropna(axis=0, inplace=True)
df_test.dropna(axis=0, inplace=True)

In [12]:
y_train = df_train['NU_NOTA_MT']

df_train.drop('NU_NOTA_MT', axis=1, inplace=True)
df_train

,NU_INSCRICAO,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,...,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,Q001,Q002,Q006,Q024,Q025,Q026,Q047
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,24,M,1,1,4,436.3,495.4,581.2,1,...,120.0,80.0,80.0,D,D,C,A,A,C,A
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,17,F,3,2,0,474.5,544.1,599.0,1,...,120.0,120.0,80.0,A,A,B,A,A,A,A
5,a37c99ec251d4f6e8ddbeabadf1c87fdbfddc4d1,18,F,1,1,1,439.7,583.2,410.9,0,...,120.0,160.0,100.0,E,E,C,B,B,A,A
6,63b4e128e9ffe8ab27f5d093db1976ef4f353e0a,18,F,3,2,0,420.1,604.2,484.5,0,...,120.0,120.0,100.0,E,D,C,B,B,C,A
7,2eb189d4912f64b19d1967e8e84b6141aba18770,18,F,1,1,1,619.6,625.8,611.2,0,...,120.0,120.0,120.0,G,F,L,B,B,A,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,aadf671ef8c1c6efa42e69432093ab83c17a52ac,20,M,1,1,3,438.2,419.7,394.5,0,...,80.0,80.0,40.0,E,E,G,B,B,B,A
13726,461f7ef2753d9d5fa0b054930d6b0e0eec318e81,33,F,3,1,10,423.8,557.9,533.0,1,...,120.0,120.0,80.0,A,A,C,A,B,B,A
13727,5354503700ecf2388f68990435390b6e1ad454ee,15,F,1,3,0,460.5,528.9,569.3,0,...,120.0,120.0,120.0,C,G,B,A,B,A,A
13728,c7a9e94a42bd76b7978f13c013a61702e36cc42c,36,M,3,4,0,422.5,621.7,569.0,1,...,80.0,80.0,100.0,B,D,G,B,B,C,A


In [13]:
df_train.dtypes

NU_INSCRICAO          object
NU_IDADE               int64
TP_SEXO               object
TP_COR_RACA            int64
TP_ST_CONCLUSAO        int64
TP_ANO_CONCLUIU        int64
NU_NOTA_CN           float64
NU_NOTA_CH           float64
NU_NOTA_LC           float64
TP_LINGUA              int64
TP_STATUS_REDACAO    float64
NU_NOTA_COMP1        float64
NU_NOTA_COMP2        float64
NU_NOTA_COMP3        float64
NU_NOTA_COMP4        float64
NU_NOTA_COMP5        float64
Q001                  object
Q002                  object
Q006                  object
Q024                  object
Q025                  object
Q026                  object
Q047                  object
dtype: object

In [14]:
to_encode = ['TP_SEXO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']

df_train = pd.get_dummies(df_train, columns=to_encode, drop_first=True)
df_test = pd.get_dummies(df_test, columns=to_encode, drop_first=True)

### Usando o Random Forest com todos os dados de test

In [15]:
features = df_train._get_numeric_data().columns.to_list()
features.insert(0,'NU_INSCRICAO')
features

['NU_INSCRICAO',
 'NU_IDADE',
 'TP_COR_RACA',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'TP_SEXO_M',
 'Q001_B',
 'Q001_C',
 'Q001_D',
 'Q001_E',
 'Q001_F',
 'Q001_G',
 'Q001_H',
 'Q002_B',
 'Q002_C',
 'Q002_D',
 'Q002_E',
 'Q002_F',
 'Q002_G',
 'Q002_H',
 'Q006_B',
 'Q006_C',
 'Q006_D',
 'Q006_E',
 'Q006_F',
 'Q006_G',
 'Q006_H',
 'Q006_I',
 'Q006_J',
 'Q006_K',
 'Q006_L',
 'Q006_M',
 'Q006_N',
 'Q006_O',
 'Q006_P',
 'Q006_Q',
 'Q024_B',
 'Q024_C',
 'Q024_D',
 'Q024_E',
 'Q025_B',
 'Q026_B',
 'Q026_C',
 'Q047_B',
 'Q047_C',
 'Q047_D',
 'Q047_E']

In [16]:
df_test = df_test[features]

In [17]:
# random forest model
mdl = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
mdl.fit(df_train.iloc[:, 1:]._get_numeric_data(), y_train)
    
    
p = mdl.predict(df_test.iloc[:,1:])

In [18]:
df_notas = pd.DataFrame(p.round(5), columns=['NU_NOTA_MT'])
df_notas.head()

,NU_NOTA_MT
0,424.597
1,438.783
2,600.622
3,551.698
4,489.624


In [19]:
df_test.reset_index(inplace=True, drop=True)

In [20]:
df_answer = pd.concat([df_test, df_notas], axis=1)
df_answer.head()

,NU_INSCRICAO,NU_IDADE,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,...,Q024_D,Q024_E,Q025_B,Q026_B,Q026_C,Q047_B,Q047_C,Q047_D,Q047_E,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,22,3,1,5,464.8,443.5,431.8,0,1.0,...,0,0,0,0,1,0,0,0,0,424.597
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,26,3,1,8,391.1,491.1,548.0,1,1.0,...,0,0,1,1,0,0,0,0,0,438.783
2,b38a03232f43b11c9d0788abaf060f7366053b6d,21,1,2,0,595.9,622.7,613.6,0,1.0,...,0,0,1,0,0,0,0,0,0,600.622
3,715494628a50142ce8cb17191cfe6d0f3cae0934,18,1,2,0,592.9,492.6,571.4,1,1.0,...,0,0,1,0,0,0,0,0,0,551.698
4,e656d6bad65c93fb2880f1eba5037008c8e75774,18,3,1,1,414.5,568.5,542.5,1,1.0,...,0,0,1,0,0,0,0,0,0,489.624


In [21]:
cols_to_keep = ['NU_INSCRICAO', 'NU_NOTA_MT']
df_answer.to_csv('answer.csv', columns=cols_to_keep, index=False)

### nota: 93.696 com get_dummies e RF

## Será que conseguimos um resultado melhor com Auto ML ?

#### Não esqueça de submeter ou salvar o resultado obtido com RF pois o arquivo answer.csv vai ser sobreescrito 

In [22]:
input("Press Enter to continue...")

Press Enter to continue...


''

In [23]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [24]:
# repetindo a celula anterior
df_test.drop(['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'IN_CEGUEIRA', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_IDOSO', 
              'TP_PRESENCA_CH','TP_PRESENCA_LC','Q027'], axis=1, inplace=True)
features = df_test.columns.to_list()
features.append('NU_NOTA_MT')
features

['NU_INSCRICAO',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'IN_TREINEIRO',
 'IN_BAIXA_VISAO',
 'IN_SURDEZ',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'TP_PRESENCA_CN',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 'Q047',
 'NU_NOTA_MT']

In [25]:
df_train = df_train[features]

df_train.isna().mean()

NU_INSCRICAO         0.000000
CO_UF_RESIDENCIA     0.000000
SG_UF_RESIDENCIA     0.000000
NU_IDADE             0.000000
TP_SEXO              0.000000
TP_COR_RACA          0.000000
TP_NACIONALIDADE     0.000000
TP_ST_CONCLUSAO      0.000000
TP_ANO_CONCLUIU      0.000000
TP_ESCOLA            0.000000
IN_TREINEIRO         0.000000
IN_BAIXA_VISAO       0.000000
IN_SURDEZ            0.000000
IN_SABATISTA         0.000000
IN_GESTANTE          0.000000
TP_PRESENCA_CN       0.000000
CO_PROVA_CN          0.000000
CO_PROVA_CH          0.000000
CO_PROVA_LC          0.000000
CO_PROVA_MT          0.000000
NU_NOTA_CN           0.246832
NU_NOTA_CH           0.246832
NU_NOTA_LC           0.261981
TP_LINGUA            0.000000
TP_STATUS_REDACAO    0.261981
NU_NOTA_COMP1        0.261981
NU_NOTA_COMP2        0.261981
NU_NOTA_COMP3        0.261981
NU_NOTA_COMP4        0.261981
NU_NOTA_COMP5        0.261981
NU_NOTA_REDACAO      0.261981
Q001                 0.000000
Q002                 0.000000
Q006      

In [26]:
# reusando more_to_drop
df_train.drop(columns=more_to_drop, inplace=True)      

In [27]:
df_train.dropna(axis=0, inplace=True)
df_test.dropna(axis=0, inplace=True)

df_train.head()

,NU_INSCRICAO,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,...,NU_NOTA_COMP4,NU_NOTA_COMP5,Q001,Q002,Q006,Q024,Q025,Q026,Q047,NU_NOTA_MT
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,24,M,1,1,4,436.3,495.4,581.2,1,...,80.0,80.0,D,D,C,A,A,C,A,399.4
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,17,F,3,2,0,474.5,544.1,599.0,1,...,120.0,80.0,A,A,B,A,A,A,A,459.8
5,a37c99ec251d4f6e8ddbeabadf1c87fdbfddc4d1,18,F,1,1,1,439.7,583.2,410.9,0,...,160.0,100.0,E,E,C,B,B,A,A,364.5
6,63b4e128e9ffe8ab27f5d093db1976ef4f353e0a,18,F,3,2,0,420.1,604.2,484.5,0,...,120.0,100.0,E,D,C,B,B,C,A,529.2
7,2eb189d4912f64b19d1967e8e84b6141aba18770,18,F,1,1,1,619.6,625.8,611.2,0,...,120.0,120.0,G,F,L,B,B,A,D,566.7


In [28]:
y_train = df_train['NU_NOTA_MT']
df_train.drop(columns='NU_NOTA_MT', inplace=True)

In [29]:
final_features = df_train.columns.to_list()
df_test = df_test[final_features]

df_inscritos = df_test['NU_INSCRICAO'].to_frame()
df_inscritos.reset_index(inplace=True)

In [30]:
df_train = pd.get_dummies(df_train.iloc[:, 1:]).values
df_test = pd.get_dummies(df_test.iloc[:, 1:]).values

In [31]:
tp = TPOTRegressor(generations=5, population_size=50, scoring='neg_mean_squared_error', verbosity=2, n_jobs=8,random_state=2020)

In [ ]:
tp.fit(df_train, y_train)

In [ ]:
tp_p = tp.predict(df_test)

In [ ]:
df_notas = pd.DataFrame(tp_p.round(5), columns=['NU_NOTA_MT'])

In [ ]:
df_notas.head()

In [ ]:
df_answer = pd.concat([df_inscritos, df_notas], axis=1)

In [ ]:
cols_to_keep = ['NU_INSCRICAO', 'NU_NOTA_MT']
df_answer.to_csv('answer.csv', columns=cols_to_keep, index=False)

## nota: 93.87

### A máquina superou o homem??? Auto ML mostrou ser um pouco melhor mas demorou quase duas horas para rodar...